In [17]:
#import importlib
%matplotlib notebook
# %matplotlib --list
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

In [18]:
# Load primitive scan_tools from ime_pylib
import scan_tools as st
# set load patp for data
st.set_path(".")

# define the default filename, just for easyines
filename="PBG_q3_empty_bulk_";
filename2 = "PBG_q3_full_bulk_";

In [19]:
# load data
res_sp_empty_lin = st.scan_load(547,578,filename=filename,top_function='measure_c_bridge_lin')
#res_sp_empty_log =st.scan_load(547,578,filename=filename,top_function='measure_c_multi_lcr')

res_sp_full_lin = st.scan_load(583,614,filename=filename2,top_function='measure_c_bridge_lin')
#res_sp_full_log =st.scan_load(583,614,filename=filename,top_function='measure_c_multi_lcr')
print(res_sp_empty_lin[0]._fieldnames)
print(res_sp_full_lin[0]._fieldnames)
#load

#res_sp=st.scan_load(35791,filename=filename)

File: .\PBG_q3_empty_bulk_000550.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
File: .\PBG_q3_empty_bulk_000552.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
File: .\PBG_q3_empty_bulk_000555.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
File: .\PBG_q3_empty_bulk_000557.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
File: .\PBG_q3_empty_bulk_000560.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
File: .\PBG_q3_empty_bulk_000562.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
File: .\PBG_q3_empty_bulk_000565.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
File: .\PBG_q3_empty_bulk_000567.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
File: .\PBG_q3_empty_bulk_000570.mat loaded . top_function: measure_c_bridge_lin (name identified from number)
F

In [20]:
def fit(f, C, model, p0, fit_method='lin'):
    
    def convert_from_complex_to_real(y):
        ''' Convert array of complex numbers to array of real numbers '''
        eps = np.finfo(float).eps  # For avoding zeros in logarithms
        if fit_method == 'lin':
            return np.abs(y)**2
        elif fit_method == 'log':
            return np.log10(np.real(y)**2+eps)+np.log10(np.imag(y)**2+eps)
        else:
            raise ValueError(f'Unknown switch = {fit_method}')
    
    def fit_func(f, *args):
        ''' The (real) fitting function '''
        C = model(f, *args)
        return convert_from_complex_to_real(C)
    
    from scipy.optimize import curve_fit
    popt, pcov = curve_fit(fit_func, f, convert_from_complex_to_real(C), p0=p0)
    
    return popt

In [97]:
m = 4.62e-3 #grams
A = 4*np.pi*(9.5e-3**2)
V = (4/3)*np.pi*(9.5e-3**3)
L = 3900#m/(A**2)

def emptymodel(f, c0, cinf, f0, q):
    x = f/f0
    return (cinf + ((c0 - cinf)/(1-(x*x)+1j*x*(1/q))))

def norm_cap(c, c0, cinf):
   return(c-cinf)/(c0-cinf)

def S_liq(f, C, popt):
    x = f/popt[2]
    F = norm_cap(C, popt[0], popt[1])
    return(L*((popt[2]*2*np.pi)**2)*((1/F)-1-1j*x*(1/popt[3])+(x**2)))

def manyfits(data):
    p0 = 2.9e-8, 1.9e-9, 9.4e4, 90
    popt = fit(data.fr_bridge, data.C_bridge, emptymodel, p0, fit_method='lin')
    C_fit = emptymodel(data.fr_bridge, *popt)
    return C_fit, popt

#print(C_fit)
#print('Fit parameters:', popt)

In [164]:
bulk_modulus = []
rhos = []
temperatures = []
fit_params = []
for empty, full in zip(res_sp_empty_lin, res_sp_full_lin):
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex = True)
    fig2, bulk = plt.subplots()
    f = empty.fr_bridge
    c = empty.C_bridge
    f_full = full.fr_bridge
    C_full = full.C_bridge
    Frequency_Full_filtered = []
    Capacitance_Full_filtered = []

    for index,Cap_full in enumerate(C_full):
        if np.real(Cap_full) < 5e-7 and np.real(Cap_full)>-1e-7:
            if f[index]>1e4 and f[index]<2e5:
                Frequency_Full_filtered.append(f[index])
                Capacitance_Full_filtered.append(Cap_full)
    
    C_fit_empty, popt_empty = manyfits(empty)
    S_full = S_liq(f_full, C_full, popt_empty)
    c0 = popt_empty[0]
    cinf = popt_empty[1]
    f0 = popt_empty[2]
    q = popt_empty[3]
    fit_params.append(popt_empty)
    Temp_start_empty = empty.T_before
    Temp_end_empty= empty.T_after
    Temp_start_full = full.T_before
    Temp_end_full = full.T_after
    
    def cm(f, k, rho):
        x = (f)/(f0)
        return cinf + (c0-cinf)/((1+(1j*x*(1/q)-x**2)+(full_s_liq(f, k, rho)/(L*(f0*2*np.pi)**2))))

    def full_s_liq(f, k, rho):
        omega_null = np.sqrt((k/rho)) * (1/9.5e-3)
        x = ([freq*2*np.pi for freq in f])/(omega_null)
        return (-1*(k/(3*(9.5e-3**3)))*(((x**2)*np.sin(x))/((x*np.cos(x))-np.sin(x))))

    popt_full = fit(Frequency_Full_filtered, Capacitance_Full_filtered, cm, (2e9, 900), fit_method = 'lin')
    C_fit_full = cm(Frequency_Full_filtered, *popt_full)

    ax1.plot(f, np.real(c), 'o')
    ax1.plot(f, np.real(C_fit_empty))
    ax1.set_ylabel(r'real $\hat{J}$', rotation = 0)
    ax2.plot(f, -1*np.imag(c), 'o')
    ax2.plot(f, -1*np.imag(C_fit_empty))
    bulk.plot(Frequency_Full_filtered, np.real(Capacitance_Full_filtered), '.', label = f'data')
    bulk.plot(Frequency_Full_filtered, np.real(C_fit_full), '--', label = f'fit')

    ax1.set_ylim([-1e-7, 2e-7])
    #ax1.set_xlim([1e4, 1.4e5])
    ax2.set_ylabel(r'$imaginary \ \hat{J}$')
    ax2.set_xlabel(r'$log(Frequency)$')

    ax1.set_xscale('log')
    bulk.set_xscale('log')
    ax1.set_title(f'Capacitance of empty Transducer at {round(Temp_start_empty)}K')
    bulk.set_title(f'Capacitance of full transducer at {round(Temp_start_full)}K')
    fig.savefig(f'EmptyCap_{round(Temp_start_empty)}K.png')
    fig2.savefig(f'FullCap_{round(Temp_start_full)}K.png')
    bulk_modulus.append(popt_full[0])
    rhos.append(popt_full[1])
    temperatures.append(Temp_start_full)
    
    
    




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<ipython-input-164-cc6428bbbb73>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1, sharex = True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [168]:
fig, (bulk, rho) = plt.subplots(2, 1 ,sharex= True)
bulk.plot(temperatures, bulk_modulus, 'o--')
rho.plot(temperatures, rhos, 'o--')
bulk.set_ylabel('Bulk modulus K')
rho.set_xlabel('Temperature Kelvin')
rho.set_ylabel(r'Density $\rho$')
bulk.set_title('Bulk modulus and Density')
fig.savefig('BulkvTemp')

<IPython.core.display.Javascript object>

In [156]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True)
ax1.plot(temperatures, [i[0] for i in fit_params], 'o--', label = 'C0')
ax2.plot(temperatures, [i[1] for i in fit_params], 'o--', label = 'Cinfinty')
ax3.plot(temperatures, [i[2] for i in fit_params], 'o--', label = 'Resonance')
ax4.plot(temperatures, [i[3] for i in fit_params], 'o--', label = 'Quality Factor')
ax1.set_ylabel('C0')
ax2.set_ylabel('Cinf')
ax3.set_ylabel('Resonance')
ax4.set_ylabel('Quality Factor')
ax3.set_xlabel('Temperature Kelvin')
ax4.set_xlabel('Temperature Kelvin')
fig.savefig('FitsvTemp')

<IPython.core.display.Javascript object>